In [11]:
%%capture
!apt-get update
!apt-get install -y libspatialindex-dev

!pip install numpy pandas tqdm --quiet
!pip install geopandas shapely pyproj rtree fiona --quiet
!pip install scikit-learn --quiet
!pip install xgboost --quiet
!pip install lightgbm --quiet
!pip install shap --quiet
!pip install matplotlib seaborn folium plotly --quiet


In [12]:
import os
import numpy as np
import pandas as pd
from pathlib import Path

from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
import folium
import plotly.express as px

import geopandas as gpd
from shapely.geometry import Point, box
from shapely.ops import nearest_points
import pyproj

from sklearn.model_selection import train_test_split, GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.ensemble import RandomForestRegressor

from xgboost import XGBRegressor
import lightgbm as lgb

import shap

import warnings
warnings.filterwarnings("ignore")


In [13]:
DATA_DIR = Path("/content")

def load_and_preprocess_csv(path):
    path = Path(path)
    if not path.exists():
        print(f"[Missing] {path.name}")
        return None

    print(f"\nLoading: {path.name}")
    df = pd.read_csv(path)

    print("Shape:", df.shape)
    print("Columns:", df.columns.tolist())
    print("\nMissing values per column:\n", df.isna().sum())

    # Detect latitude/longitude columns
    lat_cols = [c for c in df.columns if "lat" in c.lower()]
    lon_cols = [c for c in df.columns if "lon" in c.lower() or "lng" in c.lower()]

    if lat_cols and lon_cols:
        lat_col = lat_cols[0]
        lon_col = lon_cols[0]
    else:
        raise ValueError(f"No latitude/longitude columns found in {path.name}")

    # Convert to numeric
    df[lat_col] = pd.to_numeric(df[lat_col], errors="coerce")
    df[lon_col] = pd.to_numeric(df[lon_col], errors="coerce")

    # Remove rows with invalid coordinates
    df = df.dropna(subset=[lat_col, lon_col])

    # Remove impossible coordinate values
    df = df[
        (df[lat_col].between(-90, 90)) &
        (df[lon_col].between(-180, 180))
    ]

    # Drop duplicates if any
    df = df.drop_duplicates()

    print("After cleaning, shape:", df.shape)
    print("\nPreview:")
    display(df.head())

    return df


candidate_df = load_and_preprocess_csv(DATA_DIR/"candidate_grid_scores.csv")
hospitals_df = load_and_preprocess_csv(DATA_DIR/"hospitals.csv")
population_df = load_and_preprocess_csv(DATA_DIR/"population_centers.csv")
schools_df = load_and_preprocess_csv(DATA_DIR/"schools.csv")
shops_df = load_and_preprocess_csv(DATA_DIR/"shops.csv")
traffic_df = load_and_preprocess_csv(DATA_DIR/"traffic_nodes.csv")

print("\nPreprocessing completed for all datasets.")



Loading: candidate_grid_scores.csv
Shape: (10000, 10)
Columns: ['candidate_id', 'latitude', 'longitude', 'pop_within_1km', 'competitor_shops_within_0.8km', 'avg_traffic_index_within_1km', 'dist_to_nearest_hospital_km', 'dist_to_nearest_school_km', 'score', 'score_norm']

Missing values per column:
 candidate_id                     0
latitude                         0
longitude                        0
pop_within_1km                   0
competitor_shops_within_0.8km    0
avg_traffic_index_within_1km     0
dist_to_nearest_hospital_km      0
dist_to_nearest_school_km        0
score                            0
score_norm                       0
dtype: int64
After cleaning, shape: (10000, 10)

Preview:


,candidate_id,latitude,longitude,pop_within_1km,competitor_shops_within_0.8km,avg_traffic_index_within_1km,dist_to_nearest_hospital_km,dist_to_nearest_school_km,score,score_norm
0,C108578,13.115152,77.707576,120712,13,45.09,0.631,0.610,477.48300,1.000000
1,C101997,13.191919,77.507576,116702,17,22.48,0.528,0.147,458.85475,0.961298
2,C108478,13.115152,77.704545,112949,10,40.64,0.377,0.608,447.84860,0.938431
3,C101996,13.187879,77.507576,112826,16,28.66,0.338,0.590,444.00770,0.930451
4,C104972,13.090909,77.598485,109775,6,25.54,0.669,0.427,436.66835,0.915203



Loading: hospitals.csv
Shape: (1000, 7)
Columns: ['hospital_id', 'name', 'type', 'latitude', 'longitude', 'bed_count', 'emergency']

Missing values per column:
 hospital_id    0
name           0
type           0
latitude       0
longitude      0
bed_count      0
emergency      0
dtype: int64
After cleaning, shape: (1000, 7)

Preview:


,hospital_id,name,type,latitude,longitude,bed_count,emergency
0,H20000,Hospital_0,clinic,12.949296,77.567050,91,N
1,H20001,Hospital_1,clinic,12.865023,77.664311,40,Y
2,H20002,Hospital_2,hospital,13.047192,77.527843,50,N
3,H20003,Hospital_3,clinic,12.906500,77.470518,57,N
4,H20004,Hospital_4,clinic,13.168882,77.563452,50,N



Loading: population_centers.csv
Shape: (5000, 5)
Columns: ['pop_id', 'latitude', 'longitude', 'population', 'avg_income_index']

Missing values per column:
 pop_id              0
latitude            0
longitude           0
population          0
avg_income_index    0
dtype: int64
After cleaning, shape: (5000, 5)

Preview:


...


Loading: shops.csv
Shape: (8000, 7)
Columns: ['shop_id', 'name', 'type', 'latitude', 'longitude', 'avg_daily_customers', 'revenue_index']

Missing values per column:
 shop_id                0
name                   0
type                   0
latitude               0
longitude              0
avg_daily_customers    0
revenue_index          0
dtype: int64
After cleaning, shape: (8000, 7)

Preview:


,shop_id,name,type,latitude,longitude,avg_daily_customers,revenue_index
0,S100000,Shop_0,retail,12.949816,77.666080,54,739.83
1,S100001,Shop_1,retail,13.180286,77.656185,52,522.03
2,S100002,Shop_2,restaurant,13.092798,77.478726,66,461.38
3,S100003,Shop_3,retail,13.039463,77.726772,49,305.00
4,S100004,Shop_4,electronics,12.862407,77.620542,60,1810.03



Loading: traffic_nodes.csv
Shape: (4000, 5)
Columns: ['node_id', 'latitude', 'longitude', 'traffic_index', 'dist_to_highway_km']

Missing values per column:
 node_id               0
latitude              0
longitude             0
traffic_index         0
dist_to_highway_km    0
dtype: int64
After cleaning, shape: (4000, 5)

Preview:


,node_id,latitude,longitude,traffic_index,dist_to_highway_km
0,T70000,12.885008,77.477387,10.26,0.271
1,T70001,12.862857,77.459291,45.22,1.054
2,T70002,12.895421,77.689351,20.78,2.203
3,T70003,12.810542,77.545138,24.56,0.295
4,T70004,13.191389,77.733533,9.39,3.274



Preprocessing completed for all datasets.


In [14]:
shops = pd.read_csv("/content/shops.csv")
schools = pd.read_csv("/content/schools.csv")
hospitals = pd.read_csv("/content/hospitals.csv")
traffic = pd.read_csv("/content/traffic_nodes.csv")
population = pd.read_csv("/content/population_centers.csv")
candidate = pd.read_csv("/content/candidate_grid_scores.csv")



def to_gdf(df, lat_col="latitude", lon_col="longitude"):
    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df[lon_col], df[lat_col]),
        crs="EPSG:4326"
    )
    return gdf.to_crs("EPSG:3857")

shops_m = to_gdf(shops)
schools_m = to_gdf(schools)
hospitals_m = to_gdf(hospitals)
traffic_m = to_gdf(traffic)
population_m = to_gdf(population)
candidate_m = to_gdf(candidate)

print("All datasets loaded and projected into meters.")
print("Shops:", len(shops_m))
print("Schools:", len(schools_m))
print("Hospitals:", len(hospitals_m))
print("Traffic nodes:", len(traffic_m))
print("Population centers:", len(population_m))
print("Candidate grids:", len(candidate_m))


All datasets loaded and projected into meters.
Shops: 8000
Schools: 2000
Hospitals: 1000
Traffic nodes: 4000
Population centers: 5000
Candidate grids: 10000


In [15]:
import numpy as np
import geopandas as gpd
from shapely.geometry import box

all_gdfs = [
    candidate_m,
    hospitals_m,
    population_m,
    schools_m,
    shops_m,
    traffic_m
]

def compute_bounds(gdfs):
    valid = [g for g in gdfs if g is not None and len(g) > 0]
    bounds = [g.total_bounds for g in valid]
    minx = min(b[0] for b in bounds)
    miny = min(b[1] for b in bounds)
    maxx = max(b[2] for b in bounds)
    maxy = max(b[3] for b in bounds)
    return minx, miny, maxx, maxy

minx, miny, maxx, maxy = compute_bounds(all_gdfs)

CELL_SIZE = 1000

def create_grid(minx, miny, maxx, maxy, size=CELL_SIZE):
    xs = np.arange(minx, maxx + size, size)
    ys = np.arange(miny, maxy + size, size)

    polys = []
    ids = []

    for i, x in enumerate(xs):
        for j, y in enumerate(ys):
            polys.append(box(x, y, x + size, y + size))
            ids.append(f"G_{i}_{j}")

    grid = gpd.GeoDataFrame({"grid_id": ids, "geometry": polys}, crs="EPSG:3857")
    return grid.reset_index(drop=True)

grid = create_grid(minx, miny, maxx, maxy)

print("Grid created successfully.")
print("Total 1000m grid cells:", len(grid))
print("Number of columns:", len(grid.columns))
print("Bounding box (meters):")
print("Min X:", minx, "Max X:", maxx)
print("Min Y:", miny, "Max Y:", maxy)

print("\nSample rows:")
display(grid.head())


Grid created successfully.
Total 1000m grid cells: 1645
Number of columns: 2
Bounding box (meters):
Min X: 8621694.561939038 Max X: 8655090.40917702
Min Y: 1436891.8974580711 Max Y: 1482591.0604277707

Sample rows:


,grid_id,geometry
0,G_0_0,"POLYGON ((8622694.562 1436891.897, 8622694.562..."
1,G_0_1,"POLYGON ((8622694.562 1437891.897, 8622694.562..."
2,G_0_2,"POLYGON ((8622694.562 1438891.897, 8622694.562..."
3,G_0_3,"POLYGON ((8622694.562 1439891.897, 8622694.562..."
4,G_0_4,"POLYGON ((8622694.562 1440891.897, 8622694.562..."


In [16]:
from pathlib import Path
import geopandas as gpd
import numpy as np
from sklearn.neighbors import BallTree

def nearest_distance(grid_gdf, points_gdf, col_name):
    if len(points_gdf) == 0:
        grid_gdf[col_name] = 0
        return grid_gdf

    grid_xy = np.vstack([grid_gdf.geometry.centroid.x, grid_gdf.geometry.centroid.y]).T
    pts_xy  = np.vstack([points_gdf.geometry.x, points_gdf.geometry.y]).T

    tree = BallTree(pts_xy, leaf_size=40)
    dist, _ = tree.query(grid_xy, k=1)

    grid_gdf[col_name] = dist
    return grid_gdf



fe = grid.copy()

fe["centroid_x"] = fe.geometry.centroid.x
fe["centroid_y"] = fe.geometry.centroid.y

fe = nearest_distance(fe, shops_m, "dist_nearest_shop")
fe = nearest_distance(fe, schools_m, "dist_nearest_school")
fe = nearest_distance(fe, hospitals_m, "dist_nearest_hospital")
fe = nearest_distance(fe, population_m, "dist_nearest_population")
fe = nearest_distance(fe, traffic_m, "dist_nearest_traffic")



candidate_pts = candidate_m.copy()
candidate_pts["grid_id"] = gpd.sjoin_nearest(candidate_pts, fe, how="left")["grid_id"]


candidate_unique = (
    candidate_pts.sort_values("score_norm", ascending=False)
    .drop_duplicates(subset=["grid_id"])
)

candidate_clean = candidate_unique.drop(columns="geometry")



fe = fe.merge(candidate_clean, on="grid_id", how="left")
fe = fe.fillna(0)



OUTPUT_DIR = Path("/content/output")
OUTPUT_DIR.mkdir(exist_ok=True)

fe.to_file(OUTPUT_DIR / "grid_features_1000m.geojson", driver="GeoJSON")
fe.drop(columns="geometry").to_csv(OUTPUT_DIR / "grid_features_1000m.csv", index=False)


print("Phase 3 Feature Engineering Summary")
print("Total grid cells:", len(fe))
print("Unique candidate grids mapped:", (fe['score'] > 0).sum())
print("Total features:", fe.drop(columns='geometry').shape[1])

fe.head()


Phase 3 Feature Engineering Summary
Total grid cells: 1645
Unique candidate grids mapped: 1564
Total features: 18


,grid_id,geometry,centroid_x,centroid_y,dist_nearest_shop,dist_nearest_school,dist_nearest_hospital,dist_nearest_population,dist_nearest_traffic,candidate_id,latitude,longitude,pop_within_1km,competitor_shops_within_0.8km,avg_traffic_index_within_1km,dist_to_nearest_hospital_km,dist_to_nearest_school_km,score,score_norm
0,G_0_0,"POLYGON ((8622694.562 1436891.897, 8622694.562...",8.622195e+06,1.437392e+06,133.981817,253.882789,898.725490,112.663199,688.867516,C100202,12.808081,77.456061,59047.0,7.0,30.71,0.544,0.426,233.43660,0.492964
1,G_0_1,"POLYGON ((8622694.562 1437891.897, 8622694.562...",8.622195e+06,1.438392e+06,303.837398,673.007081,173.559943,303.861767,307.321155,C100204,12.816162,77.456061,58758.0,18.0,22.46,0.536,0.377,226.54205,0.478640
2,G_0_2,"POLYGON ((8622694.562 1438891.897, 8622694.562...",8.622195e+06,1.439392e+06,214.909057,265.801532,558.093517,195.498759,334.294784,C100206,12.824242,77.456061,68412.0,18.0,17.93,0.509,0.379,265.02725,0.558598
3,G_0_3,"POLYGON ((8622694.562 1439891.897, 8622694.562...",8.622195e+06,1.440392e+06,383.557749,852.179293,377.270009,350.443037,346.424248,C100207,12.828283,77.456061,70414.0,13.0,24.68,0.297,0.655,275.73875,0.580852
4,G_0_4,"POLYGON ((8622694.562 1440891.897, 8622694.562...",8.622195e+06,1.441392e+06,133.836847,264.977458,191.518866,107.325811,239.534607,C100209,12.836364,77.456061,42818.0,7.0,33.26,0.035,0.628,168.66860,0.358401


In [17]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
import numpy as np
import pandas as pd

fe = fe.drop_duplicates(subset=["grid_id"]).reset_index(drop=True)

fe_train = fe[fe["score_norm"] > 0].copy()
fe_train = fe_train.drop_duplicates(subset=["grid_id"]).reset_index(drop=True)

fe_np = fe_train.drop(columns=["geometry", "grid_id"]).select_dtypes(include=[np.number])

scaler = StandardScaler()
X = scaler.fit_transform(fe_np)

kmeans = KMeans(n_clusters=6, random_state=42)
fe_train["kmeans_cluster"] = kmeans.fit_predict(X)

dbscan = DBSCAN(eps=0.30, min_samples=5)
fe_train["dbscan_cluster"] = dbscan.fit_predict(X)

fe = fe.merge(
    fe_train[["grid_id", "kmeans_cluster", "dbscan_cluster"]],
    on="grid_id",
    how="left"
)

fe["kmeans_cluster"] = fe["kmeans_cluster"].fillna(-1).astype(int)
fe["dbscan_cluster"] = fe["dbscan_cluster"].fillna(-1).astype(int)

fe = fe.drop_duplicates(subset=["grid_id"]).reset_index(drop=True)

fe.drop(columns="geometry").to_csv("/content/output/grid_features_with_clusters.csv", index=False)
fe.to_file("/content/output/grid_features_with_clusters.geojson", driver="GeoJSON")

print("Unique grids used for clustering:", len(fe_train))
print("Total unique grids:", len(fe))
print("KMeans clusters:", fe["kmeans_cluster"].nunique())
print("DBSCAN clusters:", fe["dbscan_cluster"].nunique())
print("DBSCAN noise points (-1):", (fe["dbscan_cluster"] == -1).sum())

train_df = fe[fe["score_norm"] > 0].drop(columns=["geometry"])
test_df  = fe[fe["score_norm"] == 0].drop(columns=["geometry"])

train_df = train_df.drop_duplicates(subset=["grid_id"]).reset_index(drop=True)
test_df  = test_df.drop_duplicates(subset=["grid_id"]).reset_index(drop=True)

train_df.to_csv("/content/output/train_grids.csv", index=False)
test_df.to_csv("/content/output/test_grids.csv", index=False)

print("Training samples:", len(train_df))
print("Future prediction samples:", len(test_df))
print("Saved: train_grids.csv, test_grids.csv")

fe.head()


Unique grids used for clustering: 1564
Total unique grids: 1645
KMeans clusters: 7
DBSCAN clusters: 1
DBSCAN noise points (-1): 1645
Training samples: 1564
Future prediction samples: 81
Saved: train_grids.csv, test_grids.csv


,grid_id,geometry,centroid_x,centroid_y,dist_nearest_shop,dist_nearest_school,dist_nearest_hospital,dist_nearest_population,dist_nearest_traffic,candidate_id,...,longitude,pop_within_1km,competitor_shops_within_0.8km,avg_traffic_index_within_1km,dist_to_nearest_hospital_km,dist_to_nearest_school_km,score,score_norm,kmeans_cluster,dbscan_cluster
0,G_0_0,"POLYGON ((8622694.562 1436891.897, 8622694.562...",8.622195e+06,1.437392e+06,133.981817,253.882789,898.725490,112.663199,688.867516,C100202,...,77.456061,59047.0,7.0,30.71,0.544,0.426,233.43660,0.492964,4,-1
1,G_0_1,"POLYGON ((8622694.562 1437891.897, 8622694.562...",8.622195e+06,1.438392e+06,303.837398,673.007081,173.559943,303.861767,307.321155,C100204,...,77.456061,58758.0,18.0,22.46,0.536,0.377,226.54205,0.478640,4,-1
2,G_0_2,"POLYGON ((8622694.562 1438891.897, 8622694.562...",8.622195e+06,1.439392e+06,214.909057,265.801532,558.093517,195.498759,334.294784,C100206,...,77.456061,68412.0,18.0,17.93,0.509,0.379,265.02725,0.558598,4,-1
3,G_0_3,"POLYGON ((8622694.562 1439891.897, 8622694.562...",8.622195e+06,1.440392e+06,383.557749,852.179293,377.270009,350.443037,346.424248,C100207,...,77.456061,70414.0,13.0,24.68,0.297,0.655,275.73875,0.580852,4,-1
4,G_0_4,"POLYGON ((8622694.562 1440891.897, 8622694.562...",8.622195e+06,1.441392e+06,133.836847,264.977458,191.518866,107.325811,239.534607,C100209,...,77.456061,42818.0,7.0,33.26,0.035,0.628,168.66860,0.358401,4,-1


In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
from xgboost import XGBRegressor

df = pd.read_csv("/content/output/grid_features_with_clusters.csv")

cols_to_fix = [
    "dist_nearest_school",
    "dist_nearest_hospital",
    "dist_nearest_traffic",
    "dist_nearest_shop",
    "dist_nearest_population"
]

def clean_distance(x):
    x = str(x)
    if "Name:" in x or "\n" in x:
        try:
            return float(x.split()[1])
        except:
            return np.nan
    try:
        return float(x)
    except:
        return np.nan

for col in cols_to_fix:
    if col in df.columns:
        df[col] = df[col].apply(clean_distance)

df = df.fillna(0)

train_df = df[df["score_norm"] > 0].copy()
train_df = train_df.select_dtypes(include=[np.number])

y = train_df["score_norm"]
X = train_df.drop(columns=["score_norm"])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

rf = RandomForestRegressor(
    n_estimators=400,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

xgb = XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)

gb = GradientBoostingRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=3,
    random_state=42
)
gb.fit(X_train, y_train)
gb_pred = gb.predict(X_test)

ensemble = VotingRegressor(
    estimators=[
        ('rf', rf),
        ('xgb', xgb),
        ('gb', gb)
    ]
)
ensemble.fit(X_train, y_train)
ens_pred = ensemble.predict(X_test)

def evaluate(name, y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae  = mean_absolute_error(y_true, y_pred)
    r2   = r2_score(y_true, y_pred)
    tolerance = 0.10
    acc = np.mean(np.abs(y_true - y_pred) <= tolerance)
    print(f"\n{name}")
    print("RMSE:", rmse)
    print("MAE :", mae)
    print("R²  :", r2)
    print(f"Tolerance Accuracy:", acc*100, "%")

evaluate("Random Forest", y_test, rf_pred)
evaluate("XGBoost", y_test, xgb_pred)
evaluate("Gradient Boosting", y_test, gb_pred)
evaluate("Voting Ensemble (FINAL)", y_test, ens_pred)

model_path = "/content/output/tatasmart_final_ensemble.pkl"
joblib.dump(ensemble, model_path)

print("Model saved:", model_path)



Random Forest
RMSE: 0.001679773690407208
MAE : 0.0005416717196638694
R²  : 0.9997931149414889
Tolerance Accuracy: 100.0 %

XGBoost
RMSE: 0.004234810820046098
MAE : 0.002185202935778699
R²  : 0.9986850912814116
Tolerance Accuracy: 100.0 %

Gradient Boosting
RMSE: 0.0015652418943267869
MAE : 0.0007177334562286139
R²  : 0.9998203651879899
Tolerance Accuracy: 100.0 %

Voting Ensemble (FINAL)
RMSE: 0.0020828929388906374
MAE : 0.0008924901855019831
R²  : 0.9996819016090415
Tolerance Accuracy: 100.0 %
Model saved: /content/output/tatasmart_final_ensemble.pkl


In [19]:
import pandas as pd
import numpy as np
import joblib
import geopandas as gpd

model = joblib.load("/content/output/tatasmart_final_ensemble.pkl")

df = pd.read_csv("/content/output/grid_features_with_clusters.csv")

cols_to_fix = [
    "dist_nearest_school",
    "dist_nearest_hospital",
    "dist_nearest_traffic",
    "dist_nearest_shop",
    "dist_nearest_population"
]

def clean_distance(x):
    x = str(x)
    if "Name:" in x or "\n" in x:
        try:
            return float(x.split()[1])
        except:
            return np.nan
    try:
        return float(x)
    except:
        return np.nan

for col in cols_to_fix:
    if col in df.columns:
        df[col] = df[col].apply(clean_distance)

df = df.fillna(0)

numeric_df = df.select_dtypes(include=[np.number])
X_all = numeric_df.drop(columns=["score_norm"], errors="ignore")

df["predicted_score"] = model.predict(X_all)

df.to_csv("/content/output/predicted_grid_scores.csv", index=False)

gdf = gpd.read_file("/content/output/grid_features_1000m.geojson")
gdf = gdf.merge(df[["grid_id", "predicted_score"]], on="grid_id", how="left")
gdf.to_file("/content/output/predicted_grid_scores.geojson", driver="GeoJSON")

print("Prediction complete")
print("Saved: predicted_grid_scores.csv and predicted_grid_scores.geojson")
df.head()


Prediction complete
Saved: predicted_grid_scores.csv and predicted_grid_scores.geojson


,grid_id,centroid_x,centroid_y,dist_nearest_shop,dist_nearest_school,dist_nearest_hospital,dist_nearest_population,dist_nearest_traffic,candidate_id,latitude,...,pop_within_1km,competitor_shops_within_0.8km,avg_traffic_index_within_1km,dist_to_nearest_hospital_km,dist_to_nearest_school_km,score,score_norm,kmeans_cluster,dbscan_cluster,predicted_score
0,G_0_0,8.622195e+06,1.437392e+06,133.981817,253.882789,898.725490,112.663199,688.867516,C100202,12.808081,...,59047.0,7.0,30.71,0.544,0.426,233.43660,0.492964,4,-1,0.492976
1,G_0_1,8.622195e+06,1.438392e+06,303.837398,673.007081,173.559943,303.861767,307.321155,C100204,12.816162,...,58758.0,18.0,22.46,0.536,0.377,226.54205,0.478640,4,-1,0.478608
2,G_0_2,8.622195e+06,1.439392e+06,214.909057,265.801532,558.093517,195.498759,334.294784,C100206,12.824242,...,68412.0,18.0,17.93,0.509,0.379,265.02725,0.558598,4,-1,0.558575
3,G_0_3,8.622195e+06,1.440392e+06,383.557749,852.179293,377.270009,350.443037,346.424248,C100207,12.828283,...,70414.0,13.0,24.68,0.297,0.655,275.73875,0.580852,4,-1,0.580826
4,G_0_4,8.622195e+06,1.441392e+06,133.836847,264.977458,191.518866,107.325811,239.534607,C100209,12.836364,...,42818.0,7.0,33.26,0.035,0.628,168.66860,0.358401,4,-1,0.358243


In [20]:
!pip install folium gradio torch matplotlib --quiet

import geopandas as gpd
import pandas as pd
import folium
import gradio as gr
import html
import torch
import matplotlib.pyplot as plt
import io
import base64

# ---------------------------------------
# LOAD DATA
# ---------------------------------------
gdf = gpd.read_file("/content/output/grid_features_1000m.geojson")
gdf = gdf.to_crs(epsg=4326)

gdf["lat"] = gdf.geometry.centroid.y
gdf["lon"] = gdf.geometry.centroid.x
gdf["score_norm"] = gdf["score_norm"].fillna(0)

# ---------------------------------------
# PYTORCH COUNTS
# ---------------------------------------
green_tensor = torch.tensor(int((gdf["score_norm"] >= 0.60).sum()))
yellow_tensor = torch.tensor(int(((gdf["score_norm"] >= 0.40) & (gdf["score_norm"] < 0.60)).sum()))
red_tensor = torch.tensor(int((gdf["score_norm"] < 0.40).sum()))

pytorch_display_html = f"""
<h3>📊 PyTorch Grid Category Counts</h3>
<b>Green:</b> {green_tensor.item()}<br>
<b>Yellow:</b> {yellow_tensor.item()}<br>
<b>Red:</b> {red_tensor.item()}<br>
"""

# ---------------------------------------
# TOP-10 TABLES
# ---------------------------------------
def get_top10_tables():
    green = gdf[gdf["score_norm"] >= 0.60].nlargest(10, "score_norm")[["grid_id","score_norm"]]
    yellow = gdf[(gdf["score_norm"] >= 0.40) & (gdf["score_norm"] < 0.60)].nlargest(10, "score_norm")[["grid_id","score_norm"]]
    red = gdf[gdf["score_norm"] < 0.40].nsmallest(10, "score_norm")[["grid_id","score_norm"]]

    return (
        green.to_html(index=False),
        yellow.to_html(index=False),
        red.to_html(index=False),
    )

# ---------------------------------------
# PIE CHART
# ---------------------------------------
def generate_pie_chart():
    labels = ["Green", "Yellow", "Red"]
    sizes = [green_tensor.item(), yellow_tensor.item(), red_tensor.item()]
    colors = ["#2ecc71", "#f1c40f", "#e74c3c"]

    fig, ax = plt.subplots(figsize=(4,4))
    ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, colors=colors)
    ax.axis("equal")

    img_bytes = io.BytesIO()
    plt.savefig(img_bytes, format='png', bbox_inches='tight')
    plt.close(fig)
    img_bytes.seek(0)

    encoded = base64.b64encode(img_bytes.read()).decode()
    return f"<img src='data:image/png;base64,{encoded}' width='320'>"

# ---------------------------------------
# COLOR FUNCTION
# ---------------------------------------
def get_color(score):
    if score >= 0.60:
        return "green"
    elif score >= 0.40:
        return "yellow"
    return "red"

# ---------------------------------------
# HTML POPUP TABLE
# ---------------------------------------
def row_to_html(row):
    ignore = ["geometry", "lat", "lon"]
    html_rows = ""
    for col in row.index:
        if col not in ignore:
            html_rows += f"<tr><td><b>{col}</b></td><td>{row[col]}</td></tr>"

    return f"""
    <div style='font-size:13px'>
        <b>GRID ID:</b> {row['grid_id']}<br><br>
        <table border='1' style='border-collapse:collapse; width:300px'>
            {html_rows}
        </table>
    </div>
    """

# ---------------------------------------
# NEW AI ENGINE — 3 POINTS VERSION
# ---------------------------------------
def explain_row(row):

    score = row["score_norm"]
    pop = row.get("population_1000m", 0)
    traffic = row.get("traffic_1000m", 0)
    shops = row.get("shops_1000m", 0)
    dist_school = row.get("dist_nearest_school", 0)
    dist_hospital = row.get("dist_nearest_hospital", 0)

    explanation = f"<h3>AI Analysis — Grid {row['grid_id']}</h3>"
    explanation += f"<h4>Score: {round(score,3)}</h4>"

    # CATEGORY
    if score >= 0.60:
        explanation += "<b>🟢 High Potential Zone</b><br><br>"
    elif score >= 0.40:
        explanation += "<b>🟡 Medium Potential Zone</b><br><br>"
    else:
        explanation += "<b>🔴 Low Potential Zone</b><br><br>"

    # BASIC FEATURE SUMMARY
    explanation += "<h4>📊 Key Indicators</h4>"
    explanation += f"👨‍👩‍👧 Population: {pop}<br>"
    explanation += f"🚗 Traffic Index: {traffic}<br>"
    explanation += f"🏬 Nearby Shops: {shops}<br>"
    explanation += f"🏫 Distance to School: {int(dist_school)} m<br>"
    explanation += f"🏥 Distance to Hospital: {int(dist_hospital)} m<br>"

    # -----------------------------
    # 🚀 3 Opportunities
    # -----------------------------
    opportunities = []
    if pop > 30000:
        opportunities.append("High demand due to strong population presence.")
    if traffic > 30:
        opportunities.append("Excellent visibility — heavy traffic roads nearby.")
    if shops <= 3:
        opportunities.append("Low competition — easier dominance.")
    if dist_school < 500:
        opportunities.append("Close to school — youth customer potential.")
    if dist_hospital < 700:
        opportunities.append("Near essential hubs — consistent footfall.")

    opportunities = opportunities[:3] or ["No major strengths identified."]

    explanation += "<hr><h4>🚀 Top 3 Opportunities</h4>"
    for op in opportunities:
        explanation += f"• {op}<br>"

    # -----------------------------
    # ⚠️ 3 Drawbacks
    # -----------------------------
    drawbacks = []
    if pop < 20000:
        drawbacks.append("Low population — reduced customer flow.")
    if traffic < 15:
        drawbacks.append("Low traffic — poor visibility.")
    if shops >= 7:
        drawbacks.append("High competition — saturated market.")
    if dist_school > 900:
        drawbacks.append("Far from schools — fewer young customers.")
    if dist_hospital > 1500:
        drawbacks.append("Far from hospitals — low essential demand.")

    drawbacks = drawbacks[:3] or ["No major risks detected."]

    explanation += "<hr><h4>⚠️ Top 3 Drawbacks</h4>"
    for dr in drawbacks:
        explanation += f"• {dr}<br>"

    # -----------------------------
    # 📝 3 Suggestions
    # -----------------------------
    if score >= 0.6:
        suggestions = [
            "Proceed with expansion — strong ROI expected.",
            "Invest in branding to quickly capture the high potential zone.",
            "Consider premium store model — domain is strong."
        ]
    elif score >= 0.4:
        suggestions = [
            "Conduct a field survey to validate demand.",
            "Use introductory offers to overcome medium competition.",
            "Increase visibility with banners near traffic routes."
        ]
    else:
        suggestions = [
            "Avoid major investment — consider alternative grids.",
            "Analyze nearby green/yellow grids to relocate.",
            "Use lightweight kiosk model instead of full store."
        ]

    explanation += "<hr><h4>📝 Top 3 Suggestions</h4>"
    for sg in suggestions[:3]:
        explanation += f"• {sg}<br>"

    return explanation

# ---------------------------------------
# DRAW MAP
# ---------------------------------------
def draw_map(grid_id="", filter_color="all"):

    m = folium.Map(
        location=[gdf["lat"].mean(), gdf["lon"].mean()],
        zoom_start=11,
        tiles="OpenStreetMap",
        width="100%", height="700px"
    )

    if filter_color == "green":
        df = gdf[gdf["score_norm"] >= 0.60]
    elif filter_color == "yellow":
        df = gdf[(gdf["score_norm"] >= 0.40) & (gdf["score_norm"] < 0.60)]
    elif filter_color == "red":
        df = gdf[gdf["score_norm"] < 0.40]
    else:
        df = gdf.copy()

    for _, r in df.iterrows():
        folium.CircleMarker(
            [r["lat"], r["lon"]],
            radius=6,
            fill=True,
            fill_opacity=0.9,
            color=get_color(r["score_norm"]),
            fill_color=get_color(r["score_norm"]),
            popup=row_to_html(r)
        ).add_to(m)

    explanation = "Select a grid."

    if grid_id != "" and grid_id in gdf["grid_id"].values:
        row = gdf[gdf["grid_id"] == grid_id].iloc[0]

        folium.Marker(
            [row["lat"], row["lon"]],
            icon=folium.Icon(color="blue"),
            popup=row_to_html(row)
        ).add_to(m)

        m.location = [row["lat"], row["lon"]]
        m.zoom_start = 15

        explanation = explain_row(row)

    top_green, top_yellow, top_red = get_top10_tables()

    return (
        m._repr_html_(),
        explanation,
        pytorch_display_html,
        generate_pie_chart(),
        top_green, top_yellow, top_red
    )

# ---------------------------------------
# 🌟 GRADIO APP
# ---------------------------------------
with gr.Blocks() as app:

    gr.Markdown("# 🌡 TataSmart Grid Heatmap — Interactive Dashboard")

    pytorch_box = gr.HTML()
    pie_box = gr.HTML()

    with gr.Row():
        grid_in = gr.Textbox(label="Enter Grid ID")
        filter_color = gr.Radio(["all", "green", "yellow", "red"], value="all", label="Filter Colors")

    with gr.Row():
        map_out = gr.HTML()
        explanation_out = gr.HTML()

    gr.Markdown("### 🏆 Top 10 Grids (Side-by-Side)")

    # ---- FIXED PARALLEL VIEW ----
    with gr.Row():
        green_box = gr.HTML(label="Top 10 Green")
        yellow_box = gr.HTML(label="Top 10 Yellow")
        red_box = gr.HTML(label="Top 10 Red")

    grid_in.change(draw_map,
                   [grid_in, filter_color],
                   [map_out, explanation_out, pytorch_box, pie_box,
                    green_box, yellow_box, red_box])

    filter_color.change(draw_map,
                   [grid_in, filter_color],
                   [map_out, explanation_out, pytorch_box, pie_box,
                    green_box, yellow_box, red_box])

    # DEFAULT LOAD
    m, ex, pt, pie, g10, y10, r10 = draw_map("")
    map_out.value = m
    explanation_out.value = ex
    pytorch_box.value = pt
    pie_box.value = pie
    green_box.value = g10
    yellow_box.value = y10
    red_box.value = r10

app.launch()



It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fbc6175bc977ccfa76.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
